<center><a href="https://www.nvidia.com/ko-kr/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

# <font color="#76b900">**노트북 3: LLM 인코더 작업 실습**</font>

이전 노트북에서는 다양한 사전학습 인코더 모델을 탐색하고, 텍스트 입력이 어떻게 처리되는지를 살펴보았습니다.  
이번 노트북에서는 이 흐름을 **실제 자연어 처리 태스크에 적용**하여,  
Transformer 인코더 구조가 왜 특정 작업에 적합한지를 직접 확인합니다.

BERT는 Transformer 아키텍처 중에서도 인코더(Encoder) 구조만을 사용한 모델로,
입력 문장을 깊이 있게 이해하고, 문맥 기반의 정교한 표현을 생성하는 데 최적화되어 있습니다.
이는 문장의 일부분을 예측하거나, 입력에 포함된 정보를 분류하는 등의 이해 중심의 작업에서 강력한 성능을 발휘합니다.

반면, GPT 계열처럼 디코더(Decoder) 기반 구조는 다음 단어를 순차적으로 생성하는 데 특화되어 있어,
자유로운 문장 생성, 대화 응답 생성, 요약, 번역 등 생성 중심의 작업에 적합합니다.


#### 이번 노트북에서 다룰 주요 인코더 기반 태스크

| 태스크 유형 | 설명 | 대표 예시 |
|-------------|------|------------|
| Token Classification | 각 토큰별 예측 | 마스크 단어 복원, 품사 태깅 등 |
| Span Prediction | 문장 내 정답 범위 추론 | 질의응답 (Q&A) |
| Sequence Classification | 전체 문장의 감정/의도 분류 | 감정 분석, 뉴스 분류 |
| Zero-Shot Classification | 레이블 사전학습 없이 의미 기반 분류 | 주제 분류, 다국어 분류 등 |

## **Part 3.1:** 마스크 단어 복원 - Fill-Mask

앞서 우리는 마스킹 해제(unmasking) 파이프라인을 간단히 실행해보고,
어떻게 문장 내 가려진 단어를 예측하는지 알아봤습니다.

이번에는 이 작업을 다시 한 번 더 구체적으로 분석해보며,
모델 내부에서 실제로 어떤 일이 일어나는지 살펴보도록 하겠습니다.

In [ ]:
import warnings
warnings.filterwarnings("ignore")
from transformers import pipeline

fill_mask = pipeline("fill-mask", model="bert-base-uncased")
result = fill_mask("Hello, I'm a [MASK] model.")
print("[MASK] 복원 결과:")
for r in result:
    print(f"{r['token_str']} ({r['score']:.4f})")

Transformer Encoder는 시퀀스를 입력받아 의미론적으로 풍부한 시퀀스를 출력하는 구조입니다.  
이 구조 덕분에 `[MASK]`에 들어갈 단어를 문맥 기반으로 예측할 수 있습니다.

- 이 작업은 **Token Classification**에 해당하며,
- 전체 어휘 집합 중 하나를 예측해야 하므로 `fill-mask` 태스크로 불립니다.

## **Part 3.2:** 범위 예측 (질의응답)

앞서 살펴본 마스크 언어 모델링(Masked Language Modeling)은 문장 내 일부 토큰을 가리고, 해당 위치의 정답 토큰을 예측하는 방식이었습니다. 이러한 방식은 Transformer 인코더 구조, 특히 BERT 모델의 학습 목적과 매우 잘 부합합니다.

### **BERT의 사전 학습:**
- **목표:** 일부 토큰을 가린 후 원래의 정답 토큰을 정확히 복원
- **전략:** [MASK] 또는 랜덤 토큰으로 대체하여 문맥 기반 예측 유도
- **효과:** 각 토큰이 앞뒤 문맥을 모두 참고하는 양방향 문맥 추론 능력 학습

하지만 BERT는 단순히 마스킹된 한 단어를 예측하는 데만 쓰이지 않습니다. 모델 본체(인코더)는 문맥 이해에 최적화된 구조를 갖추고 있기 때문에, 출력 토큰을 예측하는 방식 외에도 다양한 작업에 범용적으로 활용될 수 있습니다.

### **예시: Span Prediction (범위 예측)**
- 토큰 하나를 예측하는 것이 아니라, 입력 문장 내에서 정답이 포함된 **구간(시작~끝 토큰)** 을 직접 찾아내는 방식입니다.
- 대표적으로 질의응답(Question Answering, QA) 태스크에서 사용됩니다.
- 예: 문단과 질문을 입력받고, **정답이 위치한 범위(start~end index)** 를 반환합니다.

### **인코더 구조가 Span Prediction에 적합한 이유**
- 출력 공간이 입력 시퀀스 내부로 제한되어 있어 불필요한 문장 생성을 피함
- 각 토큰의 임베딩은 문맥 정보를 포함한 벡터이므로 정확한 위치 예측이 가능
- 디코더 기반 생성 모델보다 연산량이 적고 빠름 → 자원이 제한된 환경에서도 효율적

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

# 사전 학습된 RoBERTa 모델을 사용합니다. 이 모델은 SQuAD2.0 데이터셋으로 파인튜닝되었습니다.
# SQuAD(Stanford Question Answering Dataset)는 질의응답 태스크를 위한 대표적인 데이터셋입니다.
model_name = "deepset/roberta-base-squad2"

# huggingface의 예제 코드를 참고했습니다 (https://huggingface.co/deepset/roberta-base-squad2)
# pipeline()을 통해 질의응답 태스크를 쉽게 수행할 수 있는 파이프라인을 생성합니다
# model과 tokenizer를 지정하여 RoBERTa 모델을 로드합니다
qa_pipeline = pipeline('question-answering', model=model_name)

# 질문과 문맥을 딕셔너리 형태로 입력합니다 
qa_input = {
    'question': 'Why is model conversion important?',
    'context': 'The option to convert models between FARM and transformers gives freedom to the user and let people easily switch between frameworks.'
}

# 생성된 파이프라인에 입력을 전달하여 질의응답을 수행합니다
# 모델은 문맥에서 질문에 대한 답을 찾아 반환합니다
result = qa_pipeline(qa_input)
print("\n질의응답 결과:")
print(f"Answer: {result['answer']}")

### **예측 방식 예시**
아래와 같이 `start_logits` 와 `end_logits` 가 출력되었다고 가정해봅시다:

| 토큰 위치 | start_logits | end_logits |
|-----------|--------------|------------|
| 0         | 0.1          | 0.2        |
| 1         | **2.4**      | 1.1        |
| 2         | 0.5          | **2.5**    |
| 3         | 1.0          | 0.3        |

- start_logits[1] = 2.4 → 시작 위치로 가장 가능성 높음  
- end_logits[2] = 2.5 → 종료 위치로 가장 가능성 높음  
- → 가장 확신 있는 구간: (start=1, end=2)  

이처럼 간단한 계산으로 정답 구간을 빠르게 예측할 수 있습니다.

### **이 방식의 장단점**
**장점**
- 입력의 일부분만을 출력으로 사용하므로 잘못된 텍스트 생성을 피할 수 있음
- 디코더 기반 생성 모델보다 빠르고 계산 효율이 높음
- 정확도 높은 정보 추출이 가능하여 문서 검색, 고객 QA, 법률 문서 분석 등에 적합

**단점**
- 출력이 입력 범위에 한정되므로, 새로운 문장을 생성하거나 대화 응답을 구성하는 데는 부적합


이처럼 BERT/RoBERTa 기반의 Span Prediction 모델은 **정확한 위치를 예측하는 데 최적화된 구조**이며, 질문에 대한 명확한 답변을 입력 문장에서 직접 찾아내는 데 매우 효과적입니다.

## **파트 3.3:** 시퀀스 분류 (감정 분석)

**시퀀스 분류란?**  
Transformer 인코더 기반 모델에서 전체 문장을 하나의 클래스로 분류하는 작업입니다. 대표적으로 감정 분석, 뉴스 분류, 스팸 탐지 등이 여기에 해당합니다.

BERT 모델은 마스크 언어 모델링(MLM) 외에도 사전 학습 중 다음 문장 예측(NSP, Next Sentence Prediction) 과제를 함께 수행합니다.

- **NSP란? (다음 문장 예측, Next-Sentence Prediction)**
- **목적:** 문장 A 다음에 문장 B가 실제로 이어지는 문장인지 판단
- **데이터 구성:** 절반은 실제 이어진 문장 쌍, 절반은 무관한 문장 쌍으로 구성
- **학습 효과:** 문맥 흐름 이해력 증가 및 [CLS] 토큰이 전체 의미를 대표하도록 유도

**NSP에 대한 논쟁**
- 장점: [CLS] 토큰이 문장 전체를 대표하는 능력을 키울 수 있음
- 단점: 실제 성능 향상 기여도에 대해 의문 제기

이러한 이유로, 이후에 등장한 여러 아키텍처들은 NSP를 다음과 같이 처리했습니다:  
  - ([**RoBERTa**](https://huggingface.co//transformers/model_doc/roberta) 참조): NSP 과제를 완전히 제거
  - ([**Albert**](https://huggingface.co/albert-base-v2) 문장 순서 예측(SOP)을 사용). 


결국 Transformer 인코더 기반 모델의 시퀀스 분류 헤드는 대부분 비슷한 구조를 따릅니다: **[CLS] 벡터 → 분류기(FNN)**

```python
#<s> hello world and all who live in it ... </s>  (입력 토큰)
#p_0  p_1   p_2  ...                          p_(n-1)
  |    |     |                                |
  v    v     v                                v
        [[BERT 인코더 백본]]
  |    |     |                                |
  v    v     v                                v
        [[시퀀스 분류용 분류 헤드]]
            [CLS] 벡터 → FeedForward Network → 클래스 예측
```

**이를 [텍스트 분류](https://huggingface.co/tasks/text-classification)** 혹은 시퀀스 분류라 부릅니다.

<div><img src="imgs/task-text-classification.png" alt="HuggingFace Text Classification Task" width="800"/></div>

#### 감정 분석 실습 (RoBERTa)

In [ ]:
from transformers import AutoModelForSequenceClassification

# RoBERTa 모델을 이용한 감정 분석 파이프라인 생성
# SamLowe가 개발한 'go_emotions' 데이터셋으로 학습된 RoBERTa 기반 모델을 사용
# 입력 텍스트의 감정을 28개의 세부 감정으로 분류할 수 있음
emo_model = pipeline('sentiment-analysis', 'SamLowe/roberta-base-go_emotions')

# 다양한 감정이 담긴 예시 문장들의 감정 분석 실행

# 긍정적인 감정이 담긴 문장 분석 
# "나는 내 오래된 베개가 좋아요?"
print(emo_model("I love my old pillow?")) 

# 좌절감이 담긴 문장 분석
# "왜 내가 만지는 모든 식물은 며칠 안에 죽어버리는 걸까?"  
print(emo_model("Why is it that every plant I touch dies within a few days?"))

# 혼란스러운 감정이 담긴 문장 분석 
# "이 새로운 지침들에 대해 너무 혼란스러워..."
print(emo_model("I'm so conflicted about these new instructions..."))

# 각 print문은 입력 문장에 대해 가장 높은 확률을 가진 감정 레이블과 그 확률값을 출력
# RoBERTa는 BERT를 개선한 모델로, 자연어 처리 태스크에서 우수한 성능을 보임
# go_emotions 데이터셋은 Reddit 댓글을 기반으로 만들어져 실제 온라인 대화의 감정을 잘 포착

**분류 헤드 구조 들여다보기**:

```python  
emo_model.model.classifier
```
출력 예시:
```python
RobertaClassificationHead(
  (dense): Linear(in_features=768, out_features=768, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (out_proj): Linear(in_features=768, out_features=28, bias=True)
)
```
**해설:**
- dense: 첫 번째 선형 계층 (비선형 변환)
- dropout: 과적합 방지를 위한 드롭아웃 계층
- out_proj: 최종 클래스 수(예: 28개 감정 분류)에 맞춘 출력 계층

즉, BERT나 RoBERTa 모두에서 [CLS] 토큰의 출력 벡터 하나만으로 전체 시퀀스의 의미를 대표하도록 한 후,
간단한 Feedforward 네트워크로 분류합니다.

**전체 요약**
- [CLS] 토큰은 문장의 대표 벡터
- 분류 헤드는 이 벡터 하나만으로 문장의 감정/의도/카테고리를 추론
- BERT 및 RoBERTa 기반 모델은 텍스트 분류, 감정 분석, 뉴스 기사 분류 등에서 탁월한 성능을 발휘

**참고:** 실제로 어떻게 작동하는지 보려면 공식 [`RobertaClassificationHead` 소스 코드](https://github.com/huggingface/transformers/blob/f26099e7b5cf579f99a42bab6ddd371bf2c8d548/src/transformers/models/roberta/modeling_roberta.py#L1510)를 확인하면 됩니다:

## **Part 3.4:** 제로샷 분류(Zero Shot Classification)

**제로샷 분류란?**  
사전 훈련된 언어 모델을 활용하여, 모델이 명시적으로 분류 작업에 대해 학습하지 않은 클래스에 대해서도 예측을 수행할 수 있는 분류 방식입니다.
예를 들어 ‘food’, ‘technology’ 같은 레이블에 대해 별도 분류 학습 없이도, 입력 문장이 해당 개념과 관련 있는지를 자연어 의미 수준에서 추론할 수 있습니다.

**핵심 아이디어**  
자연어 추론(NLI)을 활용해 "이 문장이 [LABEL] 주제에 관한 것이다"와 같은 방식으로 의미적 관련성을 비교합니다.

- 기존 분류 방식: 입력 → 분류기 → 고정된 클래스 예측
- 제로샷 방식: 입력 + 각 후보 레이블을 자연어 문장으로 구성 → 의미적 일치도 비교

> 즉, 구조를 바꾸지 않고도 문장을 '자연어 쿼리'로 비교하면서 다양한 클래스에 대해 예측 수행 가능

**관련 개념 정리:**
- **제로샷 추론(Zero-Shot Inference)**: 처음 보는 태스크/클래스에 대해 추론
- **퓨샷 추론(Few-Shot Inference)**: 소수의 예시만 보고 추론

#### 실습: 제로샷 감정 분류

In [ ]:
from transformers import pipeline

# 제로샷 분류 파이프라인 생성
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# 문장 입력
sentence = "I love eating spicy food at night."

# 후보 레이블
labels = ["food", "lifestyle", "technology", "education"]

# 분류 실행
result = classifier(sentence, candidate_labels=labels)

# 출력 보기
print(result)

### 결과 해석

- 출력은 각 레이블에 대해 '해당 문장이 그 주제를 내포하는가?'를 판단한 확률로 구성
- 가장 높은 확률을 갖는 레이블이 최종 예측값
- 내부적으로는 다음과 같은 비교가 이루어짐:
  - "이 문장은 food에 관한 것이다."
  - "이 문장은 technology에 관한 것이다."
  - ...

**왜 가능한가요?**  
사전학습된 LLM은 강력한 언어 이해 능력을 기반으로, 입력 문장과 각 레이블을 문장 형태로 비교하여 의미적 관련성을 추론할 수 있습니다.

**활용 예시**
- 뉴스 기사 주제 자동 분류
- 고객 리뷰 분류
- 라벨 정의 없이 빠른 태스크 프로토타이핑

# <font color="#76b900">**마무리**</font>

지금까지 우리는 트랜스포머 기반 인코더 아키텍처를 활용하여 다양한 자연어 처리 작업들을 수행하는 방법을 살펴보았습니다.
각 작업 유형은 모델이 어떻게 정보를 추론하고 활용하는지를 보여주는 좋은 예시가 됩니다.

### 정리: 주요 작업 유형

| 작업 유형                                           | 설명                                        | 대표 예시                |
| ----------------------------------------------- | ----------------------------------------- | -------------------- |
| **토큰 수준 예측**<br>(Token-Level Prediction)     | 입력 시퀀스의 **각 토큰**에 대해 개별적으로 예측을 수행         | 마스크 복원, 품사 태깅, 범위 예측 |
| **시퀀스 수준 예측**<br>(Sequence-Level Prediction) | 전체 문장을 하나의 단위로 간주하고, `[CLS]` 등의 대표 벡터로 분류 | 감정 분석, 문서 분류         |
| **다중 쿼리 예측**<br>(Multi-query Inference)      | **여러 후보 레이블**에 대해 각기 문장과의 관련도를 측정         | 제로샷 분류               |

In [ ]:
## 프로그램 종료 명령어입니다!
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<center><a href="https://www.nvidia.com/ko-kr/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>